# Import modules

In [1]:
# # Import libraries
# !pip install py_stringsimjoin
# !pip install py_stringmatching
import json, copy, os, time
import pandas as pd
import py_stringmatching as sm
import py_stringsimjoin as ssj
from py_stringmatching.tokenizer.delimiter_tokenizer import DelimiterTokenizer
from py_stringmatching.tokenizer.qgram_tokenizer import QgramTokenizer
from itertools import product

c:\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Python39\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Python39\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Read data

In [2]:
acm_df = pd.read_csv(".\dataset\library\ACM.csv", sep=',')
dblp_df = pd.read_csv(".\dataset\library\DBLP2.csv", sep=',',encoding='ISO-8859-1')
ground_truth_df = pd.read_csv(".\dataset\library\DBLP-ACM_perfectMapping.csv", sep=',')
dblp_df

,id,title,authors,venue,year
0,journals/sigmod/Mackay99,Semantic Integration of Environmental Models for Application to Global Information Systems and D...,D. Scott Mackay,SIGMOD Record,1999
1,conf/vldb/PoosalaI96,Estimation of Query-Result Distribution and its Application in Parallel-Join Load Balancing,"Viswanath Poosala, Yannis E. Ioannidis",VLDB,1996
2,conf/vldb/PalpanasSCP02,Incremental Maintenance for Non-Distributive Aggregate Functions,"Themistoklis Palpanas, Richard Sidle, Hamid Pirahesh, Roberta Cochrane",VLDB,2002
3,conf/vldb/GardarinGT96,Cost-based Selection of Path Expression Processing Algorithms in Object-Oriented Databases,"Zhao-Hui Tang, Georges Gardarin, Jean-Robert Gruser",VLDB,1996
4,conf/vldb/HoelS95,Benchmarking Spatial Join Operations with Spatial Output,"Erik G. Hoel, Hanan Samet",VLDB,1995
...,...,...,...,...,...
2611,journals/tods/KarpSP03,A simple algorithm for finding frequent elements in streams and bags,"Scott Shenker, Christos H. Papadimitriou, Richard M. Karp",ACM Trans. Database Syst.,2003
2612,conf/vldb/LimWV03,SASH: A Self-Adaptive Histogram Set for Dynamically Changing Workloads,"Lipyeow Lim, Min Wang, Jeffrey Scott Vitter",VLDB,2003
2613,journals/tods/ChakrabartiKMP02,Locally adaptive dimensionality reduction for indexing large time series databases,"Kaushik Chakrabarti, Eamonn J. Keogh, Michael J. Pazzani, Sharad Mehrotra",ACM Trans. Database Syst.,2002
2614,journals/sigmod/Snodgrass01,Chair's Message,Richard T. Snodgrass,SIGMOD Record,2001


In [3]:
acm_df

,id,title,authors,venue,year
0,304586,The WASA2 object-oriented workflow management system,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
1,304587,A user-centered interface for querying distributed multimedia databases,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999
2,304589,"World Wide Database-integrating the Web, CORBA and databases","Athman Bouguettaya, Boualem Benatallah, Lily Hendra, James Beard, Kevin Smith, Mourad Quzzani",International Conference on Management of Data,1999
3,304590,XML-based information mediation with MIX,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228;scher, Richard Marciano, Yannis Papakonstantinou,...",International Conference on Management of Data,1999
4,304582,The CCUBE constraint object-oriented database system,"Alexander Brodsky, Victor E. Segal, Jia Chen, Paval A. Exarkhopoulo",International Conference on Management of Data,1999
...,...,...,...,...,...
2289,672977,Dual-Buffering Strategies in Object Bases,"Alfons Kemper, Donald Kossmann",Very Large Data Bases,1994
2290,950482,Guest editorial,"Philip A. Bernstein, Yannis Ioannidis, Raghu Ramakrishnan",The VLDB Journal &mdash; The International Journal on Very Large Data Bases,2003
2291,672980,GraphDB: Modeling and Querying Graphs in Databases,Ralf Hartmut G&#252;ting,Very Large Data Bases,1994
2292,945741,Review of The data warehouse toolkit: the complete guide to dimensional modeling (2nd edition) b...,Alexander A. Anisimov,ACM SIGMOD Record,2003


# Data Preprocessing

In [66]:
acm_df.isna().sum()

id          0
title       0
authors    14
venue       0
year        0
dtype: int64

In [67]:
sample = acm_df[acm_df['authors'].isna()]
sample

,id,title,authors,venue,year
884,671838,"High-Performance and Scalability through Application Tier,In-Memory Data Management",NaN,Very Large Data Bases,2000
1146,673478,MineSet(tm): A System for High-End Data Mining and Visualization,NaN,Very Large Data Bases,1996
2128,959075,Database principles,NaN,ACM SIGMOD Record,2003
2129,959077,Book reviews,NaN,ACM SIGMOD Record,2003
2130,959082,Reports,NaN,ACM SIGMOD Record,2003
2131,945726,Special topic section on peer to peer data management,NaN,ACM SIGMOD Record,2003
2132,945738,Standards,NaN,ACM SIGMOD Record,2003
2133,945740,Book reviews,NaN,ACM SIGMOD Record,2003
2134,959063,Special section on sensor network technology and sensor data managment,NaN,ACM SIGMOD Record,2003
2136,758376,Author Index,NaN,Very Large Data Bases,2000


In [68]:
dblp_df.isna().sum()

id         0
title      0
authors    0
venue      0
year       0
dtype: int64

In [3]:
print(acm_df['year'].unique())
print(dblp_df['year'].unique())

print(acm_df['venue'].unique())
print(dblp_df['venue'].unique())

[1999 1998 2000 1995 1997 1996 2001 2002 1994 2003]
[1999 1996 2002 1995 1998 1997 1994 2003 2000 2001]
['International Conference on Management of Data' 'ACM SIGMOD Record '
 'ACM Transactions on Database Systems (TODS) '
 'The VLDB Journal &mdash; The International Journal on Very Large Data Bases '
 'Very Large Data Bases']
['SIGMOD Record' 'VLDB' 'SIGMOD Conference' 'VLDB J.'
 'ACM Trans. Database Syst.']


In [6]:
acm_df['title'] = acm_df['title'].str.lower()
dblp_df['title'] = dblp_df['title'].str.lower()
acm_df['authors'] = acm_df['authors'].str.lower()
dblp_df['authors'] = dblp_df['authors'].str.lower()
acm_df['venue'] = acm_df['venue'].str.lower()
dblp_df['venue'] = dblp_df['venue'].str.lower()

In [7]:
acm_df

,id,title,authors,venue,year
0,304586,the wasa2 object-oriented workflow management system,"gottfried vossen, mathias weske",international conference on management of data,1999
1,304587,a user-centered interface for querying distributed multimedia databases,"isabel f. cruz, kimberly m. james",international conference on management of data,1999
2,304589,"world wide database-integrating the web, corba and databases","athman bouguettaya, boualem benatallah, lily hendra, james beard, kevin smith, mourad quzzani",international conference on management of data,1999
3,304590,xml-based information mediation with mix,"chaitan baru, amarnath gupta, bertram lud&#228;scher, richard marciano, yannis papakonstantinou,...",international conference on management of data,1999
4,304582,the ccube constraint object-oriented database system,"alexander brodsky, victor e. segal, jia chen, paval a. exarkhopoulo",international conference on management of data,1999
...,...,...,...,...,...
2289,672977,dual-buffering strategies in object bases,"alfons kemper, donald kossmann",very large data bases,1994
2290,950482,guest editorial,"philip a. bernstein, yannis ioannidis, raghu ramakrishnan",the vldb journal &mdash; the international journal on very large data bases,2003
2291,672980,graphdb: modeling and querying graphs in databases,ralf hartmut g&#252;ting,very large data bases,1994
2292,945741,review of the data warehouse toolkit: the complete guide to dimensional modeling (2nd edition) b...,alexander a. anisimov,acm sigmod record,2003


In [8]:
acm_df.to_csv("./dataset/library/ACM_lower.csv")
dblp_df.to_csv("./dataset/library/DBLP2_lower.csv")

In [2]:
# We will use lowercased datasets from now on
acm_df = pd.read_csv(".\dataset\library\ACM_lower.csv", sep=',')
dblp_df = pd.read_csv(".\dataset\library\DBLP2_lower.csv", sep=',',encoding='ISO-8859-1')

# Define measure and tokenizer

In [3]:
TOKENIZERS = {'SPACE_DELIMITER': DelimiterTokenizer(delim_set={' '}, return_set=True),
              '2_GRAM': QgramTokenizer(qval=2, padding=False, return_set=True),
              '3_GRAM': QgramTokenizer(qval=3, padding=False, return_set=True),
              '4_GRAM': QgramTokenizer(qval=4, padding=False, return_set=True),
              '5_GRAM': QgramTokenizer(qval=5, padding=False, return_set=True),
              '6_GRAM': QgramTokenizer(qval=6, padding=False, return_set=True),
              '2_GRAM_PAD': QgramTokenizer(qval=2),
              '3_GRAM_PAD': QgramTokenizer(qval=3),
              '4_GRAM_PAD': QgramTokenizer(qval=4),
              '5_GRAM_PAD': QgramTokenizer(qval=5),
              '6_GRAM_PAD': QgramTokenizer(qval=6),
              }

# Define filters

In [11]:
FILTERS = {
    "OVERLAP_FILTER": ssj.OverlapFilter,
    "SIZE_FILTER": ssj.SizeFilter,
    "PREFIX_FILTER": ssj.PrefixFilter,
    "POSITION_FILTER": ssj.PositionFilter,
    "SUFFIX_FILTER": ssj.SuffixFilter
}

# Define string similarity functions

In [4]:
SIM_FUNC = {
    'COSINE': sm.Cosine().get_sim_score,
    'DICE': sm.Dice().get_sim_score,
    'JACCARD': sm.Jaccard().get_sim_score,
    'OVERLAP_COEFFICIENT': sm.OverlapCoefficient().get_sim_score,
    'LEVENSHTEIN': sm.Levenshtein().get_sim_score,
    'TF-IDF': sm.TfIdf().get_sim_score,
    'G_JACCARD': sm.GeneralizedJaccard().get_sim_score
}

# Define directory and load script function

In [5]:
DATA_PATH = os.sep.join([os.getcwd(), 'dataset'])
BENCHMARK_DIRECTORY = 'benchmark_results'

In [6]:
def load_scripts(scripts_file_name):
    with open(scripts_file_name, 'r') as js_file:
        scripts = json.load(js_file)['scripts']
    return scripts

# Apply the testing configuration for dataset

In [7]:
def test(script, l_table, r_table, idx_script, scripts_file_name):
    result = []
    total_info_obj = product(script['sim_funcs'], script['sim_measure_types'],
                            script['tokenizers'], script['scale_filters'],
                            script['thresholds'], script['n_jobs'])
    for sim_funcs, sim_measure_type, tokenizer, scale_filter, threshold, n_jobs in total_info_obj:
        if tokenizer in ["SPACE_DELIMITER"] and sim_measure_type != 'EDIT_DISTANCE':
            continue
        sim_func = SIM_FUNC[sim_funcs]
        tok = TOKENIZERS[tokenizer]
        if scale_filter == "OVERLAP_FILTER":
            s_filter = FILTERS[scale_filter](tok, overlap_size=1, comp_op='>=', allow_missing=False)
        else:
            s_filter = FILTERS[scale_filter](tok, sim_measure_type, threshold, allow_empty=True, allow_missing=False)
        sum_time = 0
        start_time = time.time()
        print(s_filter)
        candidate_set = s_filter.filter_tables(
            l_table, r_table,
            script['l_id_attr'], script['r_id_attr'],
            script['l_join_attr'], script['r_join_attr'],
            l_out_attrs=None, r_out_attrs=None,
            l_out_prefix='l_', r_out_prefix='r_',
            n_jobs=n_jobs, show_progress=True)
        output_table = ssj.apply_matcher(candidate_set,
                                        'l_' + script['l_id_attr'], 'r_' + script['r_id_attr'], l_table, r_table,
                                        script['l_id_attr'], script['r_id_attr'],
                                        script['l_join_attr'], script['r_join_attr'],
                                        tokenizer=tok, sim_function=sim_func, threshold=threshold,
                                        comp_op='>=', allow_missing=True,
                                        l_out_attrs=[script['l_join_attr']], r_out_attrs=[script['r_join_attr']],
                                        l_out_prefix='l_', r_out_prefix='r_',
                                        out_sim_score=True, n_jobs=n_jobs, show_progress=False)
        sum_time += (time.time() - start_time)
        cand_set_size = len(candidate_set)
        if not os.path.exists(BENCHMARK_DIRECTORY):
            os.makedirs(BENCHMARK_DIRECTORY)
        if not os.path.exists(os.sep.join([BENCHMARK_DIRECTORY, str(idx_script)])):
            os.makedirs(os.sep.join([BENCHMARK_DIRECTORY, str(idx_script)]))
        output_table.to_csv(os.sep.join([BENCHMARK_DIRECTORY, str(idx_script),
            scripts_file_name + sim_measure_type + '_' + tokenizer + str(threshold) + '_' + str(n_jobs) + scale_filter + '_' + sim_funcs + '_' + str(idx_script) + '.csv']))
        obj_res = {"tokenizer": tokenizer, "threshold": threshold, "idx_script" : idx_script, "table": copy.deepcopy(output_table), "time" : sum_time, "num_candidate": cand_set_size}
        result.append(obj_res)
    return result

In [8]:
def load_data_and_test(scripts_file_name):
    result = []
    scripts = load_scripts(scripts_file_name)
    for idx, script in enumerate(scripts):
        l_path = os.sep.join([DATA_PATH, *script['ltable']])
        r_path = os.sep.join([DATA_PATH, *script['rtable']])
        l_table = pd.read_csv(l_path, encoding=script['ltable_encoding'])
        r_table = pd.read_csv(r_path, encoding=script['rtable_encoding'])
        result.append(test(script, l_table, r_table, idx, scripts_file_name))
    return result, l_table, r_table
#result là mảng chiều: chiều thứ nhất tương ứng với chỉ số script trong list các script
#chiều thứ hai là dictionary ứng với từng lần đo trong mỗi script 

In [10]:
# Debug:
script_0 = load_scripts("./scripts/scripts_0.json")
for idx, script in enumerate(script_0):
    l_path = os.sep.join([DATA_PATH, *script['ltable']])
    r_path = os.sep.join([DATA_PATH, *script['rtable']])
    l_table = pd.read_csv(l_path, encoding=script['ltable_encoding'])
    r_table = pd.read_csv(r_path, encoding=script['rtable_encoding'])
    print(r_table)

          id  \
0     304586   
1     304587   
2     304589   
3     304590   
4     304582   
...      ...   
2289  672977   
2290  950482   
2291  672980   
2292  945741   
2293  672979   

                                                                                                    title  \
0                                                    The WASA2 object-oriented workflow management system   
1                                 A user-centered interface for querying distributed multimedia databases   
2                                            World Wide Database-integrating the Web, CORBA and databases   
3                                                                XML-based information mediation with MIX   
4                                                    The CCUBE constraint object-oriented database system   
...                                                                                                   ...   
2289                                        

In [28]:
result1_lower, l_table1_lower, r_table1_lower = load_data_and_test("./scripts/scripts_1_lower.json")

# Show an example of joined result 

### Left table, right table before join

In [13]:
l_table

,id,title,authors,venue,year
0,journals/sigmod/Mackay99,Semantic Integration of Environmental Models for Application to Global Information Systems and D...,D. Scott Mackay,SIGMOD Record,1999
1,conf/vldb/PoosalaI96,Estimation of Query-Result Distribution and its Application in Parallel-Join Load Balancing,"Viswanath Poosala, Yannis E. Ioannidis",VLDB,1996
2,conf/vldb/PalpanasSCP02,Incremental Maintenance for Non-Distributive Aggregate Functions,"Themistoklis Palpanas, Richard Sidle, Hamid Pirahesh, Roberta Cochrane",VLDB,2002
3,conf/vldb/GardarinGT96,Cost-based Selection of Path Expression Processing Algorithms in Object-Oriented Databases,"Zhao-Hui Tang, Georges Gardarin, Jean-Robert Gruser",VLDB,1996
4,conf/vldb/HoelS95,Benchmarking Spatial Join Operations with Spatial Output,"Erik G. Hoel, Hanan Samet",VLDB,1995
...,...,...,...,...,...
2611,journals/tods/KarpSP03,A simple algorithm for finding frequent elements in streams and bags,"Scott Shenker, Christos H. Papadimitriou, Richard M. Karp",ACM Trans. Database Syst.,2003
2612,conf/vldb/LimWV03,SASH: A Self-Adaptive Histogram Set for Dynamically Changing Workloads,"Lipyeow Lim, Min Wang, Jeffrey Scott Vitter",VLDB,2003
2613,journals/tods/ChakrabartiKMP02,Locally adaptive dimensionality reduction for indexing large time series databases,"Kaushik Chakrabarti, Eamonn J. Keogh, Michael J. Pazzani, Sharad Mehrotra",ACM Trans. Database Syst.,2002
2614,journals/sigmod/Snodgrass01,Chair's Message,Richard T. Snodgrass,SIGMOD Record,2001


In [14]:
r_table

,id,title,authors,venue,year
0,304586,The WASA2 object-oriented workflow management system,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
1,304587,A user-centered interface for querying distributed multimedia databases,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999
2,304589,"World Wide Database-integrating the Web, CORBA and databases","Athman Bouguettaya, Boualem Benatallah, Lily Hendra, James Beard, Kevin Smith, Mourad Quzzani",International Conference on Management of Data,1999
3,304590,XML-based information mediation with MIX,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228;scher, Richard Marciano, Yannis Papakonstantinou,...",International Conference on Management of Data,1999
4,304582,The CCUBE constraint object-oriented database system,"Alexander Brodsky, Victor E. Segal, Jia Chen, Paval A. Exarkhopoulo",International Conference on Management of Data,1999
...,...,...,...,...,...
2289,672977,Dual-Buffering Strategies in Object Bases,"Alfons Kemper, Donald Kossmann",Very Large Data Bases,1994
2290,950482,Guest editorial,"Philip A. Bernstein, Yannis Ioannidis, Raghu Ramakrishnan",The VLDB Journal &mdash; The International Journal on Very Large Data Bases,2003
2291,672980,GraphDB: Modeling and Querying Graphs in Databases,Ralf Hartmut G&#252;ting,Very Large Data Bases,1994
2292,945741,Review of The data warehouse toolkit: the complete guide to dimensional modeling (2nd edition) b...,Alexander A. Anisimov,ACM SIGMOD Record,2003


### Example of a joined-table using Position Filter, threshold 0.7

In [15]:
result[0][2]['table'] 

,_id,l_id,r_id,l_title,r_title,_sim_score
0,299,conf/sigmod/BraumandlKK99,304573,Database Patchwork on the Internet,Database patchwork on the Internet,0.771429
1,300,conf/sigmod/WolfsonSXZC99,304572,DOMINO: Databases fOr MovINg Objects tracking,DOMINO: databases fOr MovINg Objects tracking,0.826087
2,378,conf/sigmod/TrisoliniLN99,304569,Data Integration and Warehousing in Telecom Italia,Data integration and warehousing in Telecom Italia,0.709091
3,532,conf/sigmod/Brown99,304236,Implementing the Spirit of SQL-99,Implementing the spirit of SQL-99,0.764706
4,611,conf/sigmod/Mohan99,304230,A Database Perspective on Lotus Domino/Notes,A database perspective on Lotus Domino/Notes,0.708333
...,...,...,...,...,...,...
217,55621,journals/vldb/AbbadiSW01,950482,Guest editorial,Guest editorial,1.000000
218,55622,journals/vldb/BernsteinIR03,950482,Guest editorial,Guest editorial,1.000000
219,55626,conf/vldb/Guting94,672980,GraphDB: Modeling and Querying Graphs in Databases,GraphDB: Modeling and Querying Graphs in Databases,1.000000
220,55658,journals/sigmod/Anisimov03,945741,Review of The data warehouse toolkit: the complete guide to dimensional modeling (2nd edition) b...,Review of The data warehouse toolkit: the complete guide to dimensional modeling (2nd edition) b...,0.993333


# Load data from DBLP-ACM_perfectMapping.csv

* The table below is the correct matching result (ground truth table) that can be used for benchmark. 

In [14]:
import numpy as np
def caculate_true_positive(joined_table, true_label_table):
    joined_tuples = joined_table[["l_id", "r_id"]].to_records(index = False).astype(dtype=[('l_id', 'O'), ('r_id', 'O')])
    true_tuples = true_label_table[["l_id", "r_id"]].to_records(index = False).astype(dtype=[('l_id', 'O'), ('r_id', 'O')])
    return len(np.intersect1d(joined_tuples, true_tuples))
true_label_table = pd.read_csv("./dataset/library/DBLP-ACM_perfectMapping.csv", header = 0, encoding="ISO-8859-1", names = ['l_id', 'r_id'])
true_label_table

,l_id,r_id
0,conf/sigmod/SlivinskasJS01,375678
1,conf/sigmod/ChaudhuriDN01,375694
2,conf/sigmod/RinfretOO01,375669
3,conf/sigmod/BreunigKKS01,375672
4,conf/sigmod/JagadishJOT01,375687
...,...,...
2219,journals/sigmod/Scholl01,604275
2220,journals/sigmod/Rosneblatt94,190649
2221,journals/sigmod/Winslett02b,601871
2222,journals/sigmod/Labrinidis01,604283


# Encode result to .plk file and load file

In [29]:
import pickle
pickle_obj = {'result': result1_lower, 'l_table' : l_table1_lower, 'r_table':r_table1_lower}
pickle.dump(pickle_obj, open("./pkl_object/script_1_lower.pkl", "wb"))

In [15]:
import pickle
load_obj = pickle.load(open("./pkl_object/script_1_lower.pkl", "rb"))
temp_result = load_obj['result']
temp_result

[[{'tokenizer': '2_GRAM_PAD',
   'threshold': 0.1,
   'idx_script': 0,
   'table':              _id                             l_id    r_id  \
   0              0            conf/sigmod/VossenW99  304586   
   1              3  journals/sigmod/BoucelmaCGJLL02  304586   
   2              4              conf/sigmod/Eaton98  304586   
   3              5              journals/sigmod/X94  304586   
   4              7                 conf/vldb/Wang95  304586   
   ...          ...                              ...     ...   
   1160165  5374599                  conf/vldb/NgC97  672979   
   1160166  5374601         conf/sigmod/GunopulosD01  672979   
   1160167  5374603            conf/sigmod/LarsonG03  672979   
   1160168  5374605              journals/vldb/NgC98  672979   
   1160169  5374608              conf/vldb/JensenL01  672979   
   
                                                               l_title  \
   0              the wasa2 object-oriented workflow management system   


# Caculate metrics for result

We consider that result from load_data_and_test function is the prediction, and the correct matching from above table is the "ground truth labels". 

In [16]:
def calculate_metrics(result, true_label_table):
    for i in range(0, len(result)):
        for obj_res in result[i]:
            obj_res["true_positive"] = caculate_true_positive(obj_res["table"], true_label_table)
            obj_res["false_positive"] = len(obj_res["table"]) - obj_res["true_positive"]
            obj_res["false_negative"] = len(true_label_table) - obj_res["true_positive"]
            obj_res["recall"] = obj_res["true_positive"] / (obj_res["true_positive"] + obj_res["false_negative"])
            obj_res["precision"] = obj_res["true_positive"] / (obj_res["true_positive"] + obj_res["false_positive"])
            obj_res["F1"] = 2 * obj_res["precision"] * obj_res["recall"] / (obj_res["precision"] + obj_res["recall"])

In [17]:
# calculate_metrics(result, true_label_table)
calculate_metrics(temp_result, true_label_table)

# Show the caculated metrics from the scripts

In [18]:
for x in range(0, len(temp_result)):
    tokenizer_list = []
    threshold_list = []
    precision_list = []
    recall_list = []
    f1_list = []
    time_list = []
    num_candidate_list = []
    for y in range(0, len(temp_result[x])):
        tokenizer_list.append(temp_result[x][y]["tokenizer"])
        threshold_list.append(temp_result[x][y]["threshold"])
        precision_list.append(temp_result[x][y]["precision"])
        recall_list.append(temp_result[x][y]["recall"])
        f1_list.append(temp_result[x][y]["F1"])
        time_list.append(temp_result[x][y]["time"])
        num_candidate_list.append(temp_result[x][y]["num_candidate"])
        data_frame = pd.DataFrame({"tokenizer": tokenizer_list, "threshold": threshold_list, "precision": precision_list, "recall": recall_list, "f1": f1_list, "time": time_list, "num_candidate" : num_candidate_list})
    print(data_frame)
    print("\n")

     tokenizer  threshold  precision    recall        f1       time  \
0   2_GRAM_PAD        0.1   0.000479  1.000000  0.000957  60.822644   
1   2_GRAM_PAD        0.2   0.001529  0.999550  0.003053  39.381924   
2   2_GRAM_PAD        0.3   0.017792  0.997752  0.034961  20.898247   
3   2_GRAM_PAD        0.4   0.231525  0.995953  0.375710  10.677665   
4   2_GRAM_PAD        0.5   0.657424  0.991457  0.790606   6.536249   
5   2_GRAM_PAD        0.6   0.831879  0.985612  0.902243   4.783320   
6   2_GRAM_PAD        0.7   0.863781  0.977968  0.917334   2.973017   
7   2_GRAM_PAD        0.8   0.873929  0.963129  0.916364   1.837399   
8   2_GRAM_PAD        0.9   0.879645  0.936601  0.907230   0.902930   
9       2_GRAM        0.1   0.000471  1.000000  0.000943  51.210276   
10      2_GRAM        0.2   0.001400  1.000000  0.002795  32.968496   
11      2_GRAM        0.3   0.014547  0.999101  0.028676  16.254820   
12      2_GRAM        0.4   0.191397  0.996403  0.321113   8.743273   
13    

In [19]:
best_result = pd.read_csv("./benchmark_results/0/scripts/scripts_1_lower.jsonJACCARD_4_GRAM0.6_4POSITION_FILTER_JACCARD_0.csv", header = 0, encoding="ISO-8859-1", names = ['id','acm_id','dblp_id','acm_title','dblp_title','sim_score'])
best_result

,id,acm_id,dblp_id,acm_title,dblp_title,sim_score
0,0,conf/sigmod/VossenW99,304586,the wasa2 object-oriented workflow management system,the wasa2 object-oriented workflow management system,1.000000
1,6,conf/sigmod/CruzJ99,304587,a user-centered interface for querying distributed multimedia databases,a user-centered interface for querying distributed multimedia databases,1.000000
2,39,conf/sigmod/BouguettayaBH99,304589,"world wide database - integrating the web, corba, and databases","world wide database-integrating the web, corba and databases",0.721311
3,43,conf/sigmod/BaruGLMPVC99,304590,xml-based information mediation with mix,xml-based information mediation with mix,1.000000
4,47,conf/sigmod/BrodskySCE99,304582,the ccube constraint object-oriented database system,the ccube constraint object-oriented database system,1.000000
...,...,...,...,...,...,...
596,22875,journals/vldb/AbbadiSW01,950482,guest editorial,guest editorial,1.000000
597,22876,journals/vldb/Atkinson00,950482,guest editorial,guest editorial,1.000000
598,22877,conf/vldb/Guting94,672980,graphdb: modeling and querying graphs in databases,graphdb: modeling and querying graphs in databases,1.000000
599,22881,journals/sigmod/Anisimov03,945741,review of the data warehouse toolkit: the complete guide to dimensional modeling (2nd edition) b...,review of the data warehouse toolkit: the complete guide to dimensional modeling (2nd edition) b...,0.993243


In [21]:
best_result[386:391]

,id,acm_id,dblp_id,acm_title,dblp_title,sim_score
386,3209,journals/vldb/ApersCS02,505049,editorial,editorial,1.0
387,3210,journals/tods/Snodgrass01a,505049,editorial,editorial,1.0
388,3211,journals/tods/Snodgrass01,505049,editorial,editorial,1.0
389,3212,journals/vldb/ApersCS02,505055,editorial,editorial,1.0
390,3213,journals/tods/Snodgrass01a,505055,editorial,editorial,1.0


# Another Approach: Concat all the fields

### Have a look back at the data

In [22]:
acm_df

,Unnamed: 0,id,title,authors,venue,year
0,0,304586,the wasa2 object-oriented workflow management system,"gottfried vossen, mathias weske",international conference on management of data,1999
1,1,304587,a user-centered interface for querying distributed multimedia databases,"isabel f. cruz, kimberly m. james",international conference on management of data,1999
2,2,304589,"world wide database-integrating the web, corba and databases","athman bouguettaya, boualem benatallah, lily hendra, james beard, kevin smith, mourad quzzani",international conference on management of data,1999
3,3,304590,xml-based information mediation with mix,"chaitan baru, amarnath gupta, bertram lud&#228;scher, richard marciano, yannis papakonstantinou,...",international conference on management of data,1999
4,4,304582,the ccube constraint object-oriented database system,"alexander brodsky, victor e. segal, jia chen, paval a. exarkhopoulo",international conference on management of data,1999
...,...,...,...,...,...,...
2289,2289,672977,dual-buffering strategies in object bases,"alfons kemper, donald kossmann",very large data bases,1994
2290,2290,950482,guest editorial,"philip a. bernstein, yannis ioannidis, raghu ramakrishnan",the vldb journal &mdash; the international journal on very large data bases,2003
2291,2291,672980,graphdb: modeling and querying graphs in databases,ralf hartmut g&#252;ting,very large data bases,1994
2292,2292,945741,review of the data warehouse toolkit: the complete guide to dimensional modeling (2nd edition) b...,alexander a. anisimov,acm sigmod record,2003


In [23]:
dblp_df

,Unnamed: 0,id,title,authors,venue,year
0,0,journals/sigmod/Mackay99,semantic integration of environmental models for application to global information systems and d...,d. scott mackay,sigmod record,1999
1,1,conf/vldb/PoosalaI96,estimation of query-result distribution and its application in parallel-join load balancing,"viswanath poosala, yannis e. ioannidis",vldb,1996
2,2,conf/vldb/PalpanasSCP02,incremental maintenance for non-distributive aggregate functions,"themistoklis palpanas, richard sidle, hamid pirahesh, roberta cochrane",vldb,2002
3,3,conf/vldb/GardarinGT96,cost-based selection of path expression processing algorithms in object-oriented databases,"zhao-hui tang, georges gardarin, jean-robert gruser",vldb,1996
4,4,conf/vldb/HoelS95,benchmarking spatial join operations with spatial output,"erik g. hoel, hanan samet",vldb,1995
...,...,...,...,...,...,...
2611,2611,journals/tods/KarpSP03,a simple algorithm for finding frequent elements in streams and bags,"scott shenker, christos h. papadimitriou, richard m. karp",acm trans. database syst.,2003
2612,2612,conf/vldb/LimWV03,sash: a self-adaptive histogram set for dynamically changing workloads,"lipyeow lim, min wang, jeffrey scott vitter",vldb,2003
2613,2613,journals/tods/ChakrabartiKMP02,locally adaptive dimensionality reduction for indexing large time series databases,"kaushik chakrabarti, eamonn j. keogh, michael j. pazzani, sharad mehrotra",acm trans. database syst.,2002
2614,2614,journals/sigmod/Snodgrass01,chair's message,richard t. snodgrass,sigmod record,2001


### Concat all the fields (title, authors, venue, year) into one

In [24]:
acm_df = pd.read_csv(".\dataset\library\ACM_lower.csv", sep=',')
dblp_df = pd.read_csv(".\dataset\library\DBLP2_lower.csv", sep=',',encoding='ISO-8859-1')
acm_df["info"] = acm_df["title"] + " " + acm_df["authors"] + " " + acm_df["venue"] + " " + acm_df["year"].map(str)
dblp_df["info"] = dblp_df["title"] + " " + dblp_df["authors"] + " " + dblp_df["venue"] + " " + dblp_df["year"].map(str)
acm_concat_df = acm_df.drop(columns= ['title', 'authors', 'venue', 'year'])
dblp_concat_df = dblp_df.drop(columns= ['title', 'authors', 'venue', 'year'])

In [25]:
acm_concat_df = acm_concat_df.drop(acm_concat_df.columns[0], axis=1)
dblp_concat_df = dblp_concat_df.drop(dblp_concat_df.columns[0], axis=1)

In [26]:
acm_concat_df

,id,info
0,304586,"the wasa2 object-oriented workflow management system gottfried vossen, mathias weske internation..."
1,304587,"a user-centered interface for querying distributed multimedia databases isabel f. cruz, kimberly..."
2,304589,"world wide database-integrating the web, corba and databases athman bouguettaya, boualem benatal..."
3,304590,"xml-based information mediation with mix chaitan baru, amarnath gupta, bertram lud&#228;scher, r..."
4,304582,"the ccube constraint object-oriented database system alexander brodsky, victor e. segal, jia che..."
...,...,...
2289,672977,"dual-buffering strategies in object bases alfons kemper, donald kossmann very large data bases 1994"
2290,950482,"guest editorial philip a. bernstein, yannis ioannidis, raghu ramakrishnan the vldb journal &mdas..."
2291,672980,graphdb: modeling and querying graphs in databases ralf hartmut g&#252;ting very large data base...
2292,945741,review of the data warehouse toolkit: the complete guide to dimensional modeling (2nd edition) b...


### Save to csv files

In [47]:
acm_concat_df.to_csv("./dataset/library/ACM_concat_lower.csv")
dblp_concat_df.to_csv("./dataset/library/DBLP2_concat_lower.csv")

### Run the algorithms

In [48]:
'''
Paramaters:
    - Similarity Measure: Jaccard
    - Tokenizer: [2-gram-pad, 2-gram, 3-gram-pad, 3-gram, 4-gram-pad]
    - Filter: Position Filtering
    - Threshold: [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
'''
result_concat_lower, l_table_concat_lower, r_table_concat_lower = load_data_and_test("./scripts/scripts_concat_lower.json")

In [49]:
import pickle
pickle_obj = {'result': result_concat_lower, 'l_table' : l_table_concat_lower, 'r_table':r_table_concat_lower}
pickle.dump(pickle_obj, open("./pkl_object/script_concat_lower.pkl", "wb"))

In [27]:
load_obj = pickle.load(open("./pkl_object/script_concat_lower.pkl", "rb"))
temp_result = load_obj['result']
calculate_metrics(temp_result, true_label_table)

In [28]:
for x in range(0, len(temp_result)):
    tokenizer_list = []
    threshold_list = []
    TP_list = []
    FP_list = []
    FN_list = []
    precision_list = []
    recall_list = []
    f1_list = []
    time_list = []
    num_candidate_list = []
    for y in range(0, len(temp_result[x])):
        tokenizer_list.append(temp_result[x][y]["tokenizer"])
        threshold_list.append(temp_result[x][y]["threshold"])
        TP_list.append(temp_result[x][y]["true_positive"])
        FP_list.append(temp_result[x][y]["false_positive"])
        FN_list.append(temp_result[x][y]["false_negative"])
        precision_list.append(temp_result[x][y]["precision"])
        recall_list.append(temp_result[x][y]["recall"])
        f1_list.append(temp_result[x][y]["F1"])
        time_list.append(temp_result[x][y]["time"])
        num_candidate_list.append(temp_result[x][y]["num_candidate"])
        data_frame = pd.DataFrame({"tokenizer": tokenizer_list, "threshold": threshold_list, "TP": TP_list, "FP": FP_list, "FN": FN_list, "precision": precision_list, "recall": recall_list, "f1": f1_list, "time": time_list, "num_candidate" : num_candidate_list})
    print(data_frame)
    print("\n")

     tokenizer  threshold    TP     FP    FN  precision    recall        f1  \
0   2_GRAM_PAD        0.4  2213  11409    11   0.162458  0.995054  0.279313   
1   2_GRAM_PAD        0.5  2197    843    27   0.722697  0.987860  0.834726   
2   2_GRAM_PAD        0.6  2106    283   118   0.881540  0.946942  0.913072   
3   2_GRAM_PAD        0.7  1876    118   348   0.940822  0.843525  0.889521   
4   2_GRAM_PAD        0.8  1268     63   956   0.952667  0.570144  0.713361   
5   2_GRAM_PAD        0.9   323     17  1901   0.950000  0.145234  0.251950   
6       2_GRAM        0.4  2217  16127     7   0.120857  0.996853  0.215578   
7       2_GRAM        0.5  2208    997    16   0.688924  0.992806  0.813409   
8       2_GRAM        0.6  2169    350    55   0.861056  0.975270  0.914611   
9       2_GRAM        0.7  1979    143   245   0.932611  0.889838  0.910723   
10      2_GRAM        0.8  1327     67   897   0.951937  0.596673  0.733554   
11      2_GRAM        0.9   361     20  1863   0.947

# Another Approach: Concat all the fields except 'venue'

In [52]:
acm_df = pd.read_csv(".\dataset\library\ACM_lower.csv", sep=',')
dblp_df = pd.read_csv(".\dataset\library\DBLP2_lower.csv", sep=',',encoding='ISO-8859-1')
acm_df["info"] = acm_df["title"] + " " + acm_df["authors"] + " " + acm_df["year"].map(str)
dblp_df["info"] = dblp_df["title"] + " " + dblp_df["authors"] + " " + dblp_df["year"].map(str)
acm_concat1_df = acm_df.drop(columns= ['title', 'authors', 'venue', 'year'])
dblp_concat1_df = dblp_df.drop(columns= ['title', 'authors', 'venue', 'year'])

In [54]:
acm_concat1_df = acm_concat1_df.drop(acm_concat1_df.columns[0], axis=1)
dblp_concat1_df = dblp_concat1_df.drop(dblp_concat1_df.columns[0], axis=1)

### Save to csv files

In [55]:
acm_concat1_df.to_csv("./dataset/library/ACM_concat1_lower.csv")
dblp_concat1_df.to_csv("./dataset/library/DBLP2_concat1_lower.csv")

### Run the algorithm

In [56]:
'''
Paramaters:
    - Similarity Measure: Jaccard
    - Tokenizer: [2-gram-pad, 2-gram, 3-gram-pad, 3-gram, 4-gram-pad]
    - Filter: Position Filtering
    - Threshold: [0.5, 0.55, 0.6, 0.65, 0.7, 0.75]
'''
result_concat1_lower, l_table_concat1_lower, r_table_concat1_lower = load_data_and_test("./scripts/scripts_concat1_lower.json")

In [57]:
import pickle
pickle_obj = {'result': result_concat1_lower, 'l_table' : l_table_concat1_lower, 'r_table':r_table_concat1_lower}
pickle.dump(pickle_obj, open("./pkl_object/script_concat1_lower.pkl", "wb"))

In [58]:
load_obj = pickle.load(open("./pkl_object/script_concat1_lower.pkl", "rb"))
temp_result = load_obj['result']
calculate_metrics(temp_result, true_label_table)

In [59]:
for x in range(0, len(temp_result)):
    tokenizer_list = []
    threshold_list = []
    TP_list = []
    FP_list = []
    FN_list = []
    precision_list = []
    recall_list = []
    f1_list = []
    time_list = []
    num_candidate_list = []
    for y in range(0, len(temp_result[x])):
        tokenizer_list.append(temp_result[x][y]["tokenizer"])
        threshold_list.append(temp_result[x][y]["threshold"])
        TP_list.append(temp_result[x][y]["true_positive"])
        FP_list.append(temp_result[x][y]["false_positive"])
        FN_list.append(temp_result[x][y]["false_negative"])
        precision_list.append(temp_result[x][y]["precision"])
        recall_list.append(temp_result[x][y]["recall"])
        f1_list.append(temp_result[x][y]["F1"])
        time_list.append(temp_result[x][y]["time"])
        num_candidate_list.append(temp_result[x][y]["num_candidate"])
        data_frame = pd.DataFrame({"tokenizer": tokenizer_list, "threshold": threshold_list, "TP": TP_list, "FP": FP_list, "FN": FN_list, "precision": precision_list, "recall": recall_list, "f1": f1_list, "time": time_list, "num_candidate" : num_candidate_list})
    print(data_frame)
    print("\n")

     tokenizer  threshold    TP   FP   FN  precision    recall        f1  \
0   2_GRAM_PAD       0.50  2201  819   23   0.728808  0.989658  0.839436   
1   2_GRAM_PAD       0.55  2191  541   33   0.801977  0.985162  0.884181   
2   2_GRAM_PAD       0.60  2184  366   40   0.856471  0.982014  0.914956   
3   2_GRAM_PAD       0.65  2176  258   48   0.894002  0.978417  0.934307   
4   2_GRAM_PAD       0.70  2156  178   68   0.923736  0.969424  0.946029   
5   2_GRAM_PAD       0.75  2127  136   97   0.939903  0.956385  0.948072   
6       2_GRAM       0.50  2206  945   18   0.700095  0.991906  0.820837   
7       2_GRAM       0.55  2199  620   25   0.780064  0.988759  0.872100   
8       2_GRAM       0.60  2189  419   35   0.839340  0.984263  0.906043   
9       2_GRAM       0.65  2178  320   46   0.871898  0.979317  0.922490   
10      2_GRAM       0.70  2162  216   62   0.909167  0.972122  0.939591   
11      2_GRAM       0.75  2128  146   96   0.935796  0.956835  0.946198   
12  3_GRAM_P

### Show incorrect predictions

In [185]:
best_result = pd.read_csv("./benchmark_results/0/scripts/scripts_concat1_3.jsonJACCARD_2_GRAM_BAG0.66_4POSITION_FILTER_JACCARD_0.csv", header = 0, encoding="ISO-8859-1", names = ['id','acm_id','dblp_id','acm_title','dblp_title','sim_score'])
best_result.loc[best_result['dblp_id'] == 604274]

,id,acm_id,dblp_id,acm_title,dblp_title,sim_score
7,49422,journals/sigmod/Aberer02,604274,Book Review Column Karl Aberer 2002,Book review column Karl Aberer 2001,0.725000
8,49423,journals/sigmod/Aberer02b,604274,Book Review Column Karl Aberer 2002,Book review column Karl Aberer 2001,0.725000
9,49424,journals/sigmod/Aberer02a,604274,Book Review Column Karl Aberer 2002,Book review column Karl Aberer 2001,0.725000
10,49425,journals/sigmod/Aberer03d,604274,Book review column Karl Aberer 2003,Book review column Karl Aberer 2001,0.888889
11,49426,journals/sigmod/Aberer03b,604274,Book review column Karl Aberer 2003,Book review column Karl Aberer 2001,0.888889
12,49427,journals/sigmod/Aberer02c,604274,Book Review Column Karl Aberer 2002,Book review column Karl Aberer 2001,0.725000
13,49428,journals/sigmod/Aberer01a,604274,Book Review Column Karl Aberer 2001,Book review column Karl Aberer 2001,0.815789


## Improvement: apply lowercase to 'info' field before matching

In [186]:
acm_concat1_df['info'] = acm_concat1_df['info'].str.lower()
dblp_concat1_df['info'] = dblp_concat1_df['info'].str.lower()
acm_concat1_df

,id,info
0,304586,"the wasa2 object-oriented workflow management system gottfried vossen, mathias weske 1999"
1,304587,"a user-centered interface for querying distributed multimedia databases isabel f. cruz, kimberly..."
2,304589,"world wide database-integrating the web, corba and databases athman bouguettaya, boualem benatal..."
3,304590,"xml-based information mediation with mix chaitan baru, amarnath gupta, bertram lud&#228;scher, r..."
4,304582,"the ccube constraint object-oriented database system alexander brodsky, victor e. segal, jia che..."
...,...,...
2289,672977,"dual-buffering strategies in object bases alfons kemper, donald kossmann 1994"
2290,950482,"guest editorial philip a. bernstein, yannis ioannidis, raghu ramakrishnan 2003"
2291,672980,graphdb: modeling and querying graphs in databases ralf hartmut g&#252;ting 1994
2292,945741,review of the data warehouse toolkit: the complete guide to dimensional modeling (2nd edition) b...


In [187]:
acm_concat1_df.to_csv("./dataset/library/ACM_concat1_lower.csv")
dblp_concat1_df.to_csv("./dataset/library/DBLP2_concat1_lower.csv")

In [190]:
'''
Paramaters:
    - Similarity Measure: Jaccard
    - Tokenizer: [2-gram-bag, 2-gram, 3-gram-bag, 3-gram, 4-gram-bag]
    - Filter: Position Filtering
    - Threshold: [0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7]
'''
result_concat1_lower, l_table_concat1_lower, r_table_concat1_lower = load_data_and_test("./scripts/scripts_concat1_lower.json")

In [191]:
import pickle
pickle_obj = {'result': result_concat1_lower, 'l_table' : l_table_concat1_lower, 'r_table':r_table_concat1_lower}
pickle.dump(pickle_obj, open("./pkl_object/script_concat1_lower.pkl", "wb"))
load_obj = pickle.load(open("./pkl_object/script_concat1_lower.pkl", "rb"))
temp_result = load_obj['result']
calculate_metrics(temp_result, true_label_table)

for x in range(0, len(temp_result)):
    tokenizer_list = []
    threshold_list = []
    TP_list = []
    FP_list = []
    FN_list = []
    precision_list = []
    recall_list = []
    f1_list = []
    time_list = []
    num_candidate_list = []
    for y in range(0, len(temp_result[x])):
        tokenizer_list.append(temp_result[x][y]["tokenizer"])
        threshold_list.append(temp_result[x][y]["threshold"])
        TP_list.append(temp_result[x][y]["true_positive"])
        FP_list.append(temp_result[x][y]["false_positive"])
        FN_list.append(temp_result[x][y]["false_negative"])
        precision_list.append(temp_result[x][y]["precision"])
        recall_list.append(temp_result[x][y]["recall"])
        f1_list.append(temp_result[x][y]["F1"])
        time_list.append(temp_result[x][y]["time"])
        num_candidate_list.append(temp_result[x][y]["num_candidate"])
        data_frame = pd.DataFrame({"tokenizer": tokenizer_list, "threshold": threshold_list, "TP": TP_list, "FP": FP_list, "FN": FN_list, "precision": precision_list, "recall": recall_list, "f1": f1_list, "time": time_list, "num_candidate" : num_candidate_list})
    print(data_frame)
    print("\n")

    tokenizer  threshold    TP   FP  FN  precision    recall        f1  \
0  2_GRAM_BAG       0.69  2162  198  62   0.916102  0.972122  0.943281   
1  2_GRAM_BAG       0.70  2156  178  68   0.923736  0.969424  0.946029   
2  2_GRAM_BAG       0.71  2152  174  72   0.925193  0.967626  0.945934   
3  2_GRAM_BAG       0.72  2146  170  78   0.926598  0.964928  0.945374   
4  2_GRAM_BAG       0.73  2141  144  83   0.936980  0.962680  0.949656   
5  2_GRAM_BAG       0.74  2134  139  90   0.938847  0.959532  0.949077   
6  2_GRAM_BAG       0.75  2127  136  97   0.939903  0.956385  0.948072   

        time  num_candidate  
0  10.150550         165470  
1   8.122309         166612  
2   7.832255         167191  
3   8.112852         167723  
4   6.887882         165788  
5   6.870054         163498  
6   6.620936         160859  




In [11]:
best_result = pd.read_csv("./benchmark_results/0/scripts/scripts_concat1_lower.jsonJACCARD_2_GRAM_PAD0.75_4POSITION_FILTER_JACCARD_0.csv", header = 0, encoding="ISO-8859-1", names = ['id','acm_id','dblp_id','acm_title','dblp_title','sim_score'])
best_result.loc[best_result['dblp_id'] == 604274]

,id,acm_id,dblp_id,acm_title,dblp_title,sim_score
0,0,conf/sigmod/VossenW99,304586,"the wasa2 object-oriented workflow management system mathias weske, gottfried vossen 1999","the wasa2 object-oriented workflow management system gottfried vossen, mathias weske 1999",0.959459
1,72,conf/sigmod/CruzJ99,304587,"a user-centered interface for querying distributed multimedia databases kimberly m. james, isabe...","a user-centered interface for querying distributed multimedia databases isabel f. cruz, kimberly...",0.964706
2,193,conf/sigmod/BouguettayaBH99,304589,"world wide database - integrating the web, corba, and databases kevin smith, athman bouguettaya,...","world wide database-integrating the web, corba and databases athman bouguettaya, boualem benatal...",0.896552
3,208,conf/sigmod/BaruGLMPVC99,304590,"xml-based information mediation with mix yannis papakonstantinou, richard marciano, chaitanya k....","xml-based information mediation with mix chaitan baru, amarnath gupta, bertram lud&#228;scher, r...",0.858268
4,220,conf/sigmod/BrodskySCE99,304582,"the ccube constraint object-oriented database system alexander brodsky, pavel a. exarkhopoulo, v...","the ccube constraint object-oriented database system alexander brodsky, victor e. segal, jia che...",0.933962
...,...,...,...,...,...,...
561,160598,conf/vldb/KemperK94,672977,"dual-buffering strategies in object bases alfons kemper, donald kossmann 1994","dual-buffering strategies in object bases alfons kemper, donald kossmann 1994",1.000000
562,160661,journals/vldb/BernsteinIR03,950482,"guest editorial yannis e. ioannidis, raghu ramakrishnan, philip a. bernstein 2003","guest editorial philip a. bernstein, yannis ioannidis, raghu ramakrishnan 2003",0.985294
563,160737,conf/vldb/Guting94,672980,graphdb: modeling and querying graphs in databases ralf hartmut gÃÂ¼ting 1994,graphdb: modeling and querying graphs in databases ralf hartmut g&#252;ting 1994,0.857143
564,160754,journals/sigmod/Anisimov03,945741,review of the data warehouse toolkit: the complete guide to dimensional modeling (2nd edition) b...,review of the data warehouse toolkit: the complete guide to dimensional modeling (2nd edition) b...,0.985185


### For each dblp_id, we only want to choose the matching with highest similarity score

In [13]:
# Find all unique id values in DBLP2.csv
dblp_df = pd.read_csv(".\dataset\library\DBLP2.csv", sep=',',encoding='ISO-8859-1')
dblp_unique_id = dblp_df['id'].unique()
# For all rows in dataframe that has the same values of dblp_id, then only keep
# the one that has the highest similarity score
for item in dblp_unique_id:
    id_list = []
    sim_list = []
    for index, row in best_result.iterrows():
        if (item == row['acm_id']):
            id_list.append(row['id'])
            sim_list.append(row['sim_score'])
    if (len(id_list) < 2):
        continue
    else:
        max_idx = sim_list.index(max(sim_list))
        for i in range(len(sim_list)):
            if (i != max_idx):
                best_result = best_result[best_result.id != id_list[i]] 

In [14]:
best_result

,id,acm_id,dblp_id,acm_title,dblp_title,sim_score
0,0,conf/sigmod/VossenW99,304586,"the wasa2 object-oriented workflow management system mathias weske, gottfried vossen 1999","the wasa2 object-oriented workflow management system gottfried vossen, mathias weske 1999",0.959459
1,72,conf/sigmod/CruzJ99,304587,"a user-centered interface for querying distributed multimedia databases kimberly m. james, isabe...","a user-centered interface for querying distributed multimedia databases isabel f. cruz, kimberly...",0.964706
2,193,conf/sigmod/BouguettayaBH99,304589,"world wide database - integrating the web, corba, and databases kevin smith, athman bouguettaya,...","world wide database-integrating the web, corba and databases athman bouguettaya, boualem benatal...",0.896552
3,208,conf/sigmod/BaruGLMPVC99,304590,"xml-based information mediation with mix yannis papakonstantinou, richard marciano, chaitanya k....","xml-based information mediation with mix chaitan baru, amarnath gupta, bertram lud&#228;scher, r...",0.858268
4,220,conf/sigmod/BrodskySCE99,304582,"the ccube constraint object-oriented database system alexander brodsky, pavel a. exarkhopoulo, v...","the ccube constraint object-oriented database system alexander brodsky, victor e. segal, jia che...",0.933962
...,...,...,...,...,...,...
561,160598,conf/vldb/KemperK94,672977,"dual-buffering strategies in object bases alfons kemper, donald kossmann 1994","dual-buffering strategies in object bases alfons kemper, donald kossmann 1994",1.000000
562,160661,journals/vldb/BernsteinIR03,950482,"guest editorial yannis e. ioannidis, raghu ramakrishnan, philip a. bernstein 2003","guest editorial philip a. bernstein, yannis ioannidis, raghu ramakrishnan 2003",0.985294
563,160737,conf/vldb/Guting94,672980,graphdb: modeling and querying graphs in databases ralf hartmut gÃÂ¼ting 1994,graphdb: modeling and querying graphs in databases ralf hartmut g&#252;ting 1994,0.857143
564,160754,journals/sigmod/Anisimov03,945741,review of the data warehouse toolkit: the complete guide to dimensional modeling (2nd edition) b...,review of the data warehouse toolkit: the complete guide to dimensional modeling (2nd edition) b...,0.985185


In [15]:
import pickle
pickle_obj = {'result': best_result}
pickle.dump(pickle_obj, open("./pkl_object/concat_best.pkl", "wb"))
load_obj = pickle.load(open("./pkl_object/concat_best.pkl", "rb"))
filter_result = load_obj['result']
print("Predicted length: %d" % len(filter_result))
print("Ground truth length: %d" % len(true_label_table))
pred_len = len(filter_result)
gt_len = len(true_label_table)

# Merge to find true-positive pairs:
filter_result = filter_result.merge(true_label_table,
                left_on=['acm_id', 'dblp_id'], right_on = ['l_id', 'r_id'])
TP = len(filter_result)
FP = pred_len - TP
FN = gt_len - TP
Precision = TP / (TP + FP)
Recall = TP / (TP + FN)
F1 = 2 * Precision * Recall / (Precision + Recall)
print("TP: %d" % TP)
print("FP: %d" % FP)
print("FN: %d" % FN)
print("Precision: %.4f" % Precision)
print("Recall: %.4f" % Recall)
print("F1-Score: %.4f" % F1)

Predicted length: 2146
Ground truth length: 2224
TP: 2114
FP: 32
FN: 110
Precision: 0.9851
Recall: 0.9505
F1-Score: 0.9675
